# Chapter 3: DataFrames, Datasets, and Spark SQL

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.0.0`
import $ivy.`org.apache.spark::spark-hive:3.0.0`

import $ivy.$                                  

import $ivy.$                                   

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF);

import org.apache.log4j.{Level, Logger}


In [3]:
import org.apache.spark.sql.{Dataset, DataFrame, SparkSession, Row}
import org.apache.spark.sql.catalyst.expressions.aggregate._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.{Dataset, DataFrame, SparkSession, Row}

import org.apache.spark.sql.catalyst.expressions.aggregate._

import org.apache.spark.sql.expressions._

import org.apache.spark.sql.functions._

In [4]:
val spark = SparkSession
    .builder()
    .appName("chap03-01")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "512m")
    .enableHiveSupport()
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@1a56a3ef

In [5]:
import spark.implicits._

import spark.implicits._

In [6]:
val df1 = spark.read.json("data/rawpanda.json")

df1: DataFrame = [_corrupt_record: string]

In [7]:
df1.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



**JSON Example**

```
{
    "name":"mission",
    "pandas":[
        {
            "id":1,
            "zip":"94110",
            "pt":"giant",
            "happy":true,
            "attributes":[0.4,0.5]
        }
    ]
}
```

**Sample Schema Information for nested structure**

```
root
 |-- name: string (nullable = true)
 |-- pandas: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = false)
 |    |    |-- zip: string (nullable = true)
 |    |    |-- pt: string (nullable = true)
 |    |    |-- happy: boolean (nullable = false)
 |    |    |-- attributes: array (nullable = true)
 |    |    |    |-- element: double (containsNull = false)
```

In [8]:
// Case class, can used to create Dataset and print the schema
case class RawPanda(
    id: Long,
    zip: String,
    pt: String,
    happy: Boolean, 
    attributes: Array[Double]
)

case class PandaPlace(
    name: String,
    pandas: Array[RawPanda]
)

defined class RawPanda
defined class PandaPlace

**Create a Dataset with the case class**

In [8]:
// def createAndPrintSchema() = {
//     val damao = RawPanda(1, "M1B 5K7", "giant", true, Array(0.1, 0.1))
//     val pandaPlace = PandaPlace("toronro", Array(damao))
//     val df = spark.createDataFrame(Seq(pandaPlace))
//     df.printSchema()
// }

In [8]:
// createAndPrintSchema()

**Sample schema information for nested structure (.schema())—manually formatted**

```
org.apache.spark.sql.types.StructType = StructType(
    StructField(name,StringType,true),
    StructField(pandas,
        ArrayType(
            StructType(StructField(id,LongType,false),
                       StructField(zip,StringType,true),
                       StructField(pt,StringType,true),
                       StructField(happy,BooleanType,false),
                       StructField(attributes,ArrayType(DoubleType,false),true)
                      ),
            true),
        true)
```

In [9]:
val damao = RawPanda(1, "M1B 5K7", "giant", true, Array(0.1, 0.1))
val pandaPlace = PandaPlace("toronro", Array(damao))
val df = spark.createDataFrame(Seq(pandaPlace))
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- pandas: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = false)
 |    |    |-- zip: string (nullable = true)
 |    |    |-- pt: string (nullable = true)
 |    |    |-- happy: boolean (nullable = false)
 |    |    |-- attributes: array (nullable = true)
 |    |    |    |-- element: double (containsNull = false)



damao: RawPanda = RawPanda(1L, "M1B 5K7", "giant", true, Array(0.1, 0.1))
pandaPlace: PandaPlace = PandaPlace(
  "toronro",
  Array(RawPanda(1L, "M1B 5K7", "giant", true, Array(0.1, 0.1)))
)
df: DataFrame = [name: string, pandas: array<struct<id:bigint,zip:string,pt:string,happy:boolean,attributes:array<double>>>]

In [10]:
// Select and explode operators
val pandaInfo = df.explode(df("pandas")){
    case Row(pandas: Seq[Row]) =>
        pandas.map{
            case (Row(
                id: Long, 
                zip: String,
                pt: String,
                happy: Boolean,
                attrs: Seq[Double]
            )) => RawPanda(id, zip, pt, happy, attrs.toArray)
        }}

pandaInfo: DataFrame = [name: string, pandas: array<struct<id:bigint,zip:string,pt:string,happy:boolean,attributes:array<double>>> ... 5 more fields]

In [11]:
pandaInfo.printSchema()

root
 |-- name: string (nullable = true)
 |-- pandas: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: long (nullable = false)
 |    |    |-- zip: string (nullable = true)
 |    |    |-- pt: string (nullable = true)
 |    |    |-- happy: boolean (nullable = false)
 |    |    |-- attributes: array (nullable = true)
 |    |    |    |-- element: double (containsNull = false)
 |-- id: long (nullable = false)
 |-- zip: string (nullable = true)
 |-- pt: string (nullable = true)
 |-- happy: boolean (nullable = false)
 |-- attributes: array (nullable = true)
 |    |-- element: double (containsNull = false)



In [16]:
pandaInfo
    .select((pandaInfo("attributes")(0)/pandaInfo("attributes")(1))).as("squishyness")

res15: Dataset[Row] = [(attributes[0] / attributes[1]): double]

In [18]:
// If/else in Spark SQL

/**
  * Encodes pandaType to Integer values instead of String values.
  *
  * @param pandaInfo the input DataFrame
  * @return Returns a DataFrame of pandaId and integer value for pandaType.
  */
def encodePandaType(pandaInfo: DataFrame): DataFrame = {
    pandaInfo.select(
        pandaInfo("id"), 
        (when(pandaInfo("pt") === "giant", 0)
            .when(pandaInfo("pt") === "red", 1)
            .otherwise(2)).as("encodedType")
    )
}

defined function encodePandaType